In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from matplotlib import font_manager, rc
font_path = "BMJUA_ttf.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rc('axes', unicode_minus=False)

In [70]:
spec = pd.read_csv("user_spec.csv")
result = pd.read_csv("loan_result.csv")

In [71]:
merge = pd.merge(result, spec, right_on="application_id", left_on="application_id")

In [74]:
def convert_purpose(data):
    if data == "생활비" or data == "LIVING":
        return "생활비"
    elif data == "대환대출" or data == "SWITCHLOAN":
        return "대환대출"
    elif data == "주택구입" or data == "BUYHOUSE":
        return "주택구입"
    elif data == "전월세보증금" or data == "HOUSEDEPOSIT":
        return "전월세보증금"
    elif data == "사업자금" or data == "BUSINESS":
        return "사업자금"
    elif data == "투자" or data == "INVEST":
        return "투자"
    elif data == "자동차구입" or data == "BUYCAR":
        return "자동차구입"
    elif data == "기타" or data == "ETC":
        return "기타"

In [77]:
merge['purpose'] = merge['purpose'].apply(convert_purpose)
#merge_df = pd.get_dummies(data = merge, columns = ['purpose'], prefix = 'purpose')
merge

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,20220201.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527245,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527246,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527247,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527248,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,20190301.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0


In [78]:
merge.loc[merge['company_enter_month'] > 10000000, 'company_enter_month'] = round(merge['company_enter_month']/100)

In [79]:
merge['company_enter_month'].value_counts()

202201.0    517411
202112.0    419945
202111.0    411403
202110.0    383568
202202.0    336795
             ...  
191601.0         1
198402.0         1
197910.0         1
197701.0         1
198311.0         1
Name: company_enter_month, Length: 534, dtype: int64

In [80]:
def date_to_num(x):
  if str(x).find("-") != -1 :
    return str(x).split("-")[0] + str(x).split("-")[1]
  return str(x)

In [81]:
merge['loanapply_insert_time'] = merge_df['loanapply_insert_time'].apply(date_to_num)
merge['loanapply_insert_time'].value_counts()

202205    3579966
202204    3427278
202203    3262767
202206    3257239
Name: loanapply_insert_time, dtype: int64

In [82]:
def year_to_mon(x, y):
    return ((round(pd.to_numeric(y)/100 - x/100))*12 + (pd.to_numeric(y)%100 - x%100))

In [83]:
merge

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1748340,202206,7,191,42000000.0,13.6,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,202202.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
1,1748340,202206,25,169,24000000.0,17.9,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,202202.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
2,1748340,202206,2,7,24000000.0,18.5,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,202202.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
3,1748340,202206,4,268,29000000.0,10.8,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,202202.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
4,1748340,202206,11,118,5000000.0,16.4,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,202202.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527245,1428218,202206,62,200,3000000.0,14.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,201903.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527246,1428218,202206,2,7,40000000.0,11.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,201903.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527247,1428218,202206,32,257,15000000.0,7.2,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,201903.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527248,1428218,202206,33,110,44000000.0,13.5,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,201903.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0


In [84]:
merge['company_enter_month'] = year_to_mon(merge['company_enter_month'], merge['loanapply_insert_time'])

In [86]:
merge['company_enter_month'].value_counts()

3.0       548457
4.0       492949
5.0       414369
6.0       410796
7.0       353092
           ...  
470.0          1
1275.0         1
521.0          1
510.0          1
545.0          1
Name: company_enter_month, Length: 542, dtype: int64

In [88]:
merge.rename(columns={'company_enter_month' : 'working_period'}, inplace=True)
merge

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,working_period,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1748340,202206,7,191,42000000.0,13.6,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,4.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
1,1748340,202206,25,169,24000000.0,17.9,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,4.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
2,1748340,202206,2,7,24000000.0,18.5,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,4.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
3,1748340,202206,4,268,29000000.0,10.8,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,4.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
4,1748340,202206,11,118,5000000.0,16.4,NaN,430982,1996.0,1.0,2022-06-07 13:05:39,620.0,24000000.0,EARNEDINCOME,4.0,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527245,1428218,202206,62,200,3000000.0,14.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,39.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527246,1428218,202206,2,7,40000000.0,11.8,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,39.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527247,1428218,202206,32,257,15000000.0,7.2,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,39.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0
13527248,1428218,202206,33,110,44000000.0,13.5,NaN,478802,1965.0,1.0,2022-06-03 12:01:47,610.0,45000000.0,EARNEDINCOME,39.0,정규직,기타가족소유,50000000.0,대환대출,0.0,NaN,2.0,56000000.0


In [89]:
merge.to_csv("merge.csv", index=False)

In [90]:
merge.isnull().sum()

application_id                                0
loanapply_insert_time                         0
bank_id                                       0
product_id                                    0
loan_limit                                 7382
loan_rate                                  7382
is_applied                              3257239
user_id                                       0
birth_year                               128096
gender                                   128096
insert_time                                   0
credit_score                            1509276
yearly_income                                 6
income_type                                   0
working_period                           400337
employment_type                               0
houseown_type                                 0
desired_amount                                0
purpose                                       0
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    1